In [136]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import ensemble
import re

#Import Data
train_dat = pd.read_csv("train.csv", encoding = "ISO-8859-1", low_memory=False)
test_dat = pd.read_csv("test.csv")
train_dat_cl = train_dat.dropna(axis=0, subset=["compliance"])

In [24]:
#Overall Compliance Rate
train_dat["compliance"] = train_dat["compliance"].fillna(-1)
numb_compl = sum(train_dat["compliance"] == 1)
numb_non_compl = sum(train_dat["compliance"] == 0)
numb_not_responsible = sum(train_dat["compliance"] == -1)
compl_rate = numb_compl/(numb_compl + numb_non_compl)
print("Compliance:", numb_compl, "Non-Compliance:", numb_non_compl, "Not Responsible:", numb_not_responsible)
print("Compliance Rate:", compl_rate)

Compliance: 11597 Non-Compliance: 148283 Not Responsible: 90426
Compliance Rate: 0.0725356517388


In [142]:
#Detroit vs Non-Detroit Mailing Address as new feature
train_dat_cl["city"] = [x.lower() for x in train_dat_cl["city"]]
train_dat_cl["inDetroit"] = np.zeros(train_dat_cl.shape[0])
r = re.compile("det")
vmatch = np.vectorize(lambda x:bool(r.match(x)))
vmatch(train_dat_cl["city"])
train_dat_cl.loc[vmatch(train_dat_cl["city"]), "inDetroit"] = 1.
print(train_dat_cl[["city", "inDetroit"]])

C:\Users\Dominic\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Dominic\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


                    city  inDetroit
0                chicago        0.0
1                detroit        1.0
5              log beach        0.0
6                clinton        0.0
7                detroit        1.0
8                detroit        1.0
9                detroit        1.0
12               detroit        1.0
13               detroit        1.0
14               detroit        1.0
18               detroit        1.0
19               detroit        1.0
21               detroit        1.0
22               detroit        1.0
23               detroit        1.0
26               detroit        1.0
28                   det        1.0
30                   det        1.0
31                   det        1.0
33          w.bloomfield        0.0
34                   det        1.0
35             southgate        0.0
37               detroit        1.0
38                   det        1.0
39               detroit        1.0
40                   det        1.0
41                   det    

C:\Users\Dominic\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [104]:
inDetroit_dat = train_dat_cl[train_dat_cl["inDetroit"] == 1]
nonDetroit_dat = train_dat_cl[train_dat_cl["inDetroit"] == 0]

print("Compliance Rate in Detroit", sum(inDetroit_dat["compliance"] == 1)/inDetroit_dat.shape[0])
print("Compliance Rate Outside Detroit", sum(nonDetroit_dat["compliance"] == 1)/nonDetroit_dat.shape[0])


# clf = ensemble.RandomForestClassifier()
# clf.fit(train_dat_cl["city"], train_dat_cl["compliance"])
# y_pred = clf.predict(test_dat["city"])
# y_true = test_dat["compliance"]
# metrics.roc_auc_score(y_true, y_pred)

Compliance Rate in Detroit 0.0716930326489
Compliance Rate Outside Detroit 0.0738716302537


In [169]:
train_dat_cl['zip_payment_rate'] = np.zeros(train_dat_cl.shape[0])
for zip_code in train_dat_cl.zip_code.unique()[1:200]:
    zip_dat = train_dat_cl[train_dat_cl["zip_code"] == zip_code]
    if zip_dat.shape[0] >= 4:
        comp_rate = sum(zip_dat["compliance"] == 1)/zip_dat.shape[0]
        print(zip_code, comp_rate)
        if comp_rate <= 0.065:
            train_dat_cl.loc[train_dat_cl['zip_code'] == zip_code, 'zip_payment_rate'] = 1.
        elif comp_rate > 0.065 and comp_rate <= 0.09:
            train_dat_cl.loc[train_dat_cl['zip_code'] == zip_code, 'zip_payment_rate'] = 2.
        else:
            train_dat_cl.loc[train_dat_cl['zip_code'] == zip_code, 'zip_payment_rate'] = 3.


C:\Users\Dominic\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Dominic\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


48208 0.0546415981199
48038 0.0872093023256
48211 0.0729431721798
48205 0.0560453400504
48213 0.0575035063114
48235 0.0647471499562
48215 0.0725190839695
48229 0.0392156862745
48210 0.0821302534488
48224 0.0577384246449
48227 0.0635593220339
48202 0.0642307692308
48221 0.0738943574102
48238 0.0651333946642
48322 0.0928819444444
48195 0.110344827586
48226 0.0948560641812
48219 0.0658723404255
48207 0.0928237129485
48230 0.110694183865
48034 0.0630213160334
48303 0.0526315789474
48044 0.087962962963
48126 0.100529100529
48209 0.12204007286
48228 0.0682044473946
48098 0.141176470588
48302 0.089552238806
48111 0.0708446866485
48141 0.105919003115
48091 0.0746268656716
48075 0.0607699650016
48214 0.0593509820666
48234 0.0877495686468
48312 0.163265306122
48240 0.0767543859649
48231 0.0549450549451
48138 0.157894736842
48203 0.0676883780332
48076 0.0669322709163
48206 0.0598905845091
48236 0.090297790586
48201 0.105004101723
48223 0.0660651920838
48204 0.0750516410374
89134 0.0434782608696
4

In [170]:
print(train_dat_cl[['zip_code', 'zip_payment_rate']])
print(sum(train_dat_cl['zip_payment_rate'] == 0.))
print(sum(train_dat_cl['zip_payment_rate'] == 1.))
print(sum(train_dat_cl['zip_payment_rate'] == 2.))
print(sum(train_dat_cl['zip_payment_rate'] == 3.))

         zip_code  zip_payment_rate
0           60606               0.0
1           48208               1.0
5       908041512               0.0
6           48038               2.0
7           48211               2.0
8           48211               2.0
9           48205               1.0
12          48213               1.0
13          48235               1.0
14          48213               1.0
18          48215               2.0
19          48215               2.0
21          48229               1.0
22          48210               2.0
23          48224               1.0
26          48227               1.0
28          48202               1.0
30          48221               2.0
31          48238               2.0
33          48322               3.0
34          48235               1.0
35          48195               3.0
37          48224               1.0
38          48221               2.0
39          48226               3.0
40          48219               2.0
41          48207           